# Advent of Code, 8th Dec 2024

## Resonant Collinearity - Part 1

You find yourselves on the roof of a top-secret Easter Bunny installation.

While The Historians do their thing, you take a look at the familiar **huge antenna**. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand Imitation Mediocre Chocolate as a Christmas gift! Unthinkable!

Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific **frequency** indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:

```
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
```

The signal only applies its nefarious effect at specific **antinodes** based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency a, they create the two antinodes marked with #:

```
..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
```
Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
```

Antennas with different frequencies don't create antinodes; A and a count as different frequencies. However, antinodes **can** occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital A creates no antinodes but has a lowercase-a-frequency antinode at its location:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
```

The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost A-frequency antenna:

```
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
```

Because the topmost A-frequency antenna overlaps with a 0-frequency antinode, there are **14** total unique locations that contain an antinode within the bounds of the map.

Calculate the impact of the signal. **How many unique locations within the bounds of the map contain an antinode?**

### Read in the data

In [1]:
#!F#
let inputFilePath = "aoc-2024-12-08-puzzle-input.txt"

Read the lines from the file.

In [2]:
open System.IO

let lines: string list = File.ReadAllLines(inputFilePath) |> Array.toList
lines.Length

50

### Solve the problem

We have to convert the lines into positions of transmitters of different frequencies.  Each frequency is represented by a different character (excluding '.').

In [3]:
lines |> List.map (fun str -> str.Length) |> Set.ofList

Count,1
(values),[ 50 ]


We see that the lines define a 50x50 square.

In [4]:
let N = lines.Length
N

50

In [5]:
let identity n = n

let numberedLines = Seq.zip (Seq.initInfinite identity) (lines |> List.rev) |> Seq.toList
numberedLines[0]

Item1,0
Item2,........w...............XC.......G................


In [6]:
let transmitterPostionsSeq = seq {
    for row, line in numberedLines do
        let numberedChars: (int*char) list = Seq.zip (Seq.initInfinite identity) line |> Seq.toList
        for col, chr in numberedChars do
            if (chr <> '.') then yield (chr, col, row) // x, y
}
let transmitterPostionsList = transmitterPostionsSeq |> Seq.toList

let transmitterFrequencies = transmitterPostionsList |> List.map (fun (chr, col, row) -> chr) |> Set.ofList
transmitterFrequencies

Count,54
(values),"[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, G, H, K, L ... (34 more) ]"


In [13]:
let rec buildTransmitterMap (transmitterFrequenciesList: char list) (positionsList: (char*int*int) list) (map: Map<char,(int*int) list>): Map<char, (int*int) list> =
    match transmitterFrequenciesList with
    | [] -> map
    | head::tail ->
        let newMap = map |> Map.add head (transmitterPostionsList |> List.filter (fun (chr, col, row) -> chr = head) |> List.map (fun (chr, col, row) -> (col, row)))
        newMap |> buildTransmitterMap tail positionsList

let transmitterMap = Map.empty<char,(int*int) list> |> buildTransmitterMap (transmitterFrequencies |> Set.toList) transmitterPostionsList
transmitterMap.Count

54

Let's see how many transmitters there are for each frequency.

In [15]:
transmitterMap.Keys |> Seq.map (fun (key:char) -> (key, transmitterMap[key].Length))

[ (0, 4), (1, 3), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4), (9, 4), (A, 4), (B, 4), (C, 4), (D, 4), (E, 4), (F, 4), (G, 4), (H, 4), (K, 4), (L, 4) ... (more) ] f Microsoft.FSharp.Collections.SeqModule+revamp@729[System.Char,System.Tuple`2[System.Char,System.Int32]] f Microsoft.FSharp.Collections.SeqModule+Map@749-1[System.Char,System.Tuple`2[System.Char,System.Int32]] mapping FSI_0023+it@1-1 ie [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, G, H, K, L ... (34 more) ] (values) [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, G, H, K, L ... (34 more) ] (values) index value 0 (0, 4) Item1 0 Item2 4 1 (1, 3) Item1 1 Item2 3 2 (2, 4) Item1 2 Item2 4 3 (3, 4) Item1 3 Item2 4 4 (4, 4) Item1 4 Item2 4 5 (5, 4) Item1 5 Item2 4 6 (6, 4) Item1 6 Item2 4 7 (7, 4) Item1 7 Item2 4 8 (8, 4) Item1 8 Item2 4 9 (9, 4) Item1 9 Item2 4 10 (A, 4) Item1 A Item2 4 11 (B, 4) Item1 B Item2 4 12 (C, 4) Item1 C Item2 4 13 (D, 4) Item1 D Item2 4 14 (E, 4) Item1 E Item2 4 15 (F, 4) Item1 F Item2 4 16 (G, 4) Item1 G Item2 4 17 (H, 4) Item1 H Item2 4 18 (K, 4) Item1 K Item2 4 19 (L, 4) Item1 L Item2 4 ... (more)

Now let's look for valid antinodes.

In [17]:
let isValidLocation (colrow: int*int): Boolean =
    let col, row = colrow
    (col >= 0) && (col < N) && (row >= 0) && (row < N)

In [19]:
let rec getPairs (aList: 'a list): ('a*'a) list =
    match aList with
    | [] -> []
    | head::tail ->
        (tail |> List.map (fun item -> (head,item))) @ (getPairs tail)

In [21]:
getPairs [1;2;3;4]

[ (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4) ] HeadOrDefault (1, 2) Item1 1 Item2 2 TailOrNull [ (1, 3), (1, 4), (2, 3), (2, 4), (3, 4) ] HeadOrDefault (1, 3) Item1 1 Item2 3 TailOrNull [ (1, 4), (2, 3), (2, 4), (3, 4) ] HeadOrDefault (1, 4) Item1 1 Item2 4 TailOrNull [ (2, 3), (2, 4), (3, 4) ] HeadOrDefault (2, 3) Item1 2 Item2 3 TailOrNull [ (2, 4), (3, 4) ] HeadOrDefault (2, 4) Item1 2 Item2 4 TailOrNull [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) Head (2, 4) Item1 2 Item2 4 Tail [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) (values) index value 0 (2, 4) Item1 2 Item2 4 1 (3, 4) Item1 3 Item2 4 Head (2, 3) Item1 2 Item2 3 Tail [ (2, 4), (3, 4) ] HeadOrDefault (2, 4) Item1 2 Item2 4 TailOrNull [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) Head (2, 4) Item1 2 Item2 4 Tail [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) (values) index value 0 (2, 4) Item1 2 Item2 4 1 (3, 4) Item1 3 Item2 4 (values) index value 0 (2, 3) Item1 2 Item2 3 1 (2, 4) Item1 2 Item2 4 2 (3, 4) Item1 3 Item2 4 Head (1, 4) Item1 1 Item2 4 Tail [ (2, 3), (2, 4), (3, 4) ] HeadOrDefault (2, 3) Item1 2 Item2 3 TailOrNull [ (2, 4), (3, 4) ] HeadOrDefault (2, 4) Item1 2 Item2 4 TailOrNull [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) Head (2, 4) Item1 2 Item2 4 Tail [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) (values) index value 0 (2, 4) Item1 2 Item2 4 1 (3, 4) Item1 3 Item2 4 Head (2, 3) Item1 2 Item2 3 Tail [ (2, 4), (3, 4) ] HeadOrDefault (2, 4) Item1 2 Item2 4 TailOrNull [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) Head (2, 4) Item1 2 Item2 4 Tail [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) (values) index value 0 (2, 4) Item1 2 Item2 4 1 (3, 4) Item1 3 Item2 4 (values) index value 0 (2, 3) Item1 2 Item2 3 1 (2, 4) Item1 2 Item2 4 2 (3, 4) Item1 3 Item2 4 (values) index value 0 (1, 4) Item1 1 Item2 4 1 (2, 3) Item1 2 Item2 3 2 (2, 4) Item1 2 Item2 4 3 (3, 4) Item1 3 Item2 4 Head (1, 3) Item1 1 Item2 3 Tail [ (1, 4), (2, 3), (2, 4), (3, 4) ] HeadOrDefault (1, 4) Item1 1 Item2 4 TailOrNull [ (2, 3), (2, 4), (3, 4) ] HeadOrDefault (2, 3) Item1 2 Item2 3 TailOrNull [ (2, 4), (3, 4) ] HeadOrDefault (2, 4) Item1 2 Item2 4 TailOrNull [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) Head (2, 4) Item1 2 Item2 4 Tail [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) (values) index value 0 (2, 4) Item1 2 Item2 4 1 (3, 4) Item1 3 Item2 4 Head (2, 3) Item1 2 Item2 3 Tail [ (2, 4), (3, 4) ] HeadOrDefault (2, 4) Item1 2 Item2 4 TailOrNull [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) Head (2, 4) Item1 2 Item2 4 Tail [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) (values) index value 0 (2, 4) Item1 2 Item2 4 1 (3, 4) Item1 3 Item2 4 (values) index value 0 (2, 3) Item1 2 Item2 3 1 (2, 4) Item1 2 Item2 4 2 (3, 4) Item1 3 Item2 4 Head (1, 4) Item1 1 Item2 4 Tail [ (2, 3), (2, 4), (3, 4) ] HeadOrDefault (2, 3) Item1 2 Item2 3 TailOrNull [ (2, 4), (3, 4) ] HeadOrDefault (2, 4) Item1 2 Item2 4 TailOrNull [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) Head (2, 4) Item1 2 Item2 4 Tail [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] (values) index value 0 (3, 4) (values) index value 0 (2, 4) Item1 2 Item2 4 1 (3, 4) Item1 3 Item2 4 Head (2, 3) Item1 2 Item2 3 Tail [ (2, 4), (3, 4) ] HeadOrDefault (2, 4) Item1 2 Item2 4 TailOrNull [ (3, 4) ] HeadOrDefault (3, 4) TailOrNull [ ] Head (3, 4) Tail [ ] 

In [25]:
let getValidAntinodes (location1: int*int) (location2: int*int): (int*int) list =
    let col1, row1 = location1
    let col2, row2 = location2
    let antinode1 = (col1*2-col2, row1*2-row2)
    let antinode2 = (col2*2-col1, row2*2-row1)
    [antinode1; antinode2] |> List.filter isValidLocation

In [24]:
getValidAntinodes (3,2) (4,3)

[ (2, 1), (5, 4) ] HeadOrDefault (2, 1) Item1 2 Item2 1 TailOrNull [ (5, 4) ] HeadOrDefault (5, 4) Item1 5 Item2 4 TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4142\r\n at lambda_method484(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.556801\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 57a9edab-ee61-61ca-19e6-f818cb833bb4 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140711575153696 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4142
 at lambda_method484(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4147\r\n at lambda_method485(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663766 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.556801\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 57a9edab-ee61-61ca-19e6-f818cb833bb4 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140711575153712 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList<T> Retu

In [28]:
let validAntinodeSeq = seq {
    for frequency in transmitterMap.Keys do
        let locations = transmitterMap[frequency]
        let locationPairs = getPairs locations
        for locationPair in locationPairs do
            let location1, location2 = locationPair
            for validAntinode in getValidAntinodes location1 location2 do
                yield validAntinode
}
let validAntinodeSet = validAntinodeSeq |> Set.ofSeq
validAntinodeSet.Count

392

## Resonant Collinearity - Part 2

Watching over your shoulder as you work, one of The Historians asks if you took the effects of resonant harmonics into your calculations.

Whoops!

After updating your model, it turns out that an antinode occurs at **any grid position** exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).

So, these three T-frequency antennas now create many antinodes:

```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```

In fact, the three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to **9**.

The original example now has **34** antinodes, including the antinodes that appear on every antenna:

```
##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
```

Calculate the impact of the signal using this updated model. **How many unique locations within the bounds of the map contain an antinode?**

### Solve the problem

In [36]:
let getValidAntinodes2 (location1: int*int) (location2: int*int): (int*int) list =
    let col1, row1 = location1
    let col2, row2 = location2
    let antinodeSeq = seq {
        yield location1
        yield location2
        for idx in 2..N do
            let antinode1 = (col1*idx-col2*(idx-1), row1*idx-row2*(idx-1))
            if (isValidLocation antinode1) then yield antinode1
            let antinode2 = (col2*idx-col1*(idx-1), row2*idx-row1*(idx-1))
            if (isValidLocation antinode2) then yield antinode2
    }
    antinodeSeq |> Set.ofSeq |> Set.toList

In [38]:
getValidAntinodes2 (3,2) (4,3)

[ (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19) ... (more) ] HeadOrDefault (1, 0) Item1 1 Item2 0 TailOrNull [ (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20) ... (more) ] HeadOrDefault (2, 1) Item1 2 Item2 1 TailOrNull [ (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21) ... (more) ] HeadOrDefault (3, 2) Item1 3 Item2 2 TailOrNull [ (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22) ... (more) ] HeadOrDefault (4, 3) Item1 4 Item2 3 TailOrNull [ (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23) ... (more) ] HeadOrDefault (5, 4) Item1 5 Item2 4 TailOrNull [ (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23), (25, 24) ... (more) ] HeadOrDefault (6, 5) TailOrNull [ (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23), (25, 24), (26, 25) ... (more) ] Head (6, 5) Tail [ (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23), (25, 24), (26, 25) ... (more) ] (values) index value 0 (6, 5) 1 (7, 6) 2 (8, 7) 3 (9, 8) 4 (10, 9) 5 (11, 10) 6 (12, 11) 7 (13, 12) 8 (14, 13) 9 (15, 14) 10 (16, 15) 11 (17, 16) 12 (18, 17) 13 (19, 18) 14 (20, 19) 15 (21, 20) 16 (22, 21) 17 (23, 22) 18 (24, 23) 19 (25, 24) ... (more) Head (5, 4) Item1 5 Item2 4 Tail [ (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23), (25, 24) ... (more) ] HeadOrDefault (6, 5) TailOrNull [ (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23), (25, 24), (26, 25) ... (more) ] Head (6, 5) Tail [ (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23), (25, 24), (26, 25) ... (more) ] (values) index value 0 (6, 5) 1 (7, 6) 2 (8, 7) 3 (9, 8) 4 (10, 9) 5 (11, 10) 6 (12, 11) 7 (13, 12) 8 (14, 13) 9 (15, 14) 10 (16, 15) 11 (17, 16) 12 (18, 17) 13 (19, 18) 14 (20, 19) 15 (21, 20) 16 (22, 21) 17 (23, 22) 18 (24, 23) 19 (25, 24) ... (more) (values) index value 0 (5, 4) Item1 5 Item2 4 1 (6, 5) Item1 6 Item2 5 2 (7, 6) Item1 7 Item2 6 3 (8, 7) Item1 8 Item2 7 4 (9, 8) Item1 9 Item2 8 5 (10, 9) Item1 10 Item2 9 6 (11, 10) Item1 11 Item2 10 7 (12, 11) Item1 12 Item2 11 8 (13, 12) Item1 13 Item2 12 9 (14, 13) Item1 14 Item2 13 10 (15, 14) Item1 15 Item2 14 11 (16, 15) Item1 16 Item2 15 12 (17, 16) Item1 17 Item2 16 13 (18, 17) Item1 18 Item2 17 14 (19, 18) Item1 19 Item2 18 15 (20, 19) Item1 20 Item2 19 16 (21, 20) Item1 21 Item2 20 17 (22, 21) Item1 22 Item2 21 18 (23, 22) Item1 23 Item2 22 19 (24, 23) Item1 24 Item2 23 ... (more) Head (4, 3) Item1 4 Item2 3 Tail [ (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23) ... (more) ] HeadOrDefault (5, 4) Item1 5 Item2 4 TailOrNull [ (6, 

In [40]:
let validAntinodeSeq2 = seq {
    for frequency in transmitterMap.Keys do
        let locations = transmitterMap[frequency]
        let locationPairs = getPairs locations
        for locationPair in locationPairs do
            let location1, location2 = locationPair
            for validAntinode in getValidAntinodes2 location1 location2 do
                yield validAntinode
}
let validAntinodeSet2 = validAntinodeSeq2 |> Set.ofSeq
validAntinodeSet2.Count

1235